In [1]:
import os,glob


In [2]:
import numpy as np

In [3]:
import librosa

In [4]:
import torch

In [5]:
import soundfile as sf

In [20]:
root_input = '/home/data2/kbh/AVTR/extract/'
sr      = 16000
n_fft   = 512
n_shift = 128
n_overlap = n_fft - n_shift
n_mels  = 40

time_shift = (1/16000)*n_shift

n_frame       = 600
n_frame_shift = 300

mel_basis = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=n_mels)

In [7]:
list_data = [x for x in glob.glob(os.path.join(root_input,'wav','*.wav'))]

In [24]:
def process(idx):
    path_target = list_data[idx]
    id_data = path_target.split('/')[-1]
    id_data = id_data.split('.')[0]
    path_label = os.path.join(root_input,'label',id_data+'.npy')
   
    ## Load
    npy_label = np.load(path_label)
    raw,_     = librosa.load(path_target,sr=sr)
    

    ## Data
    spec = librosa.stft(raw,window='hann',n_fft=n_fft,hop_length=n_shift, win_length=None,center=True,dtype=np.cdouble)
    mel = np.matmul(mel_basis,np.abs(spec))
    pt = torch.from_numpy(mel)
    
    print(np.shape(npy_label))
    print(np.shape(mel))

    # Label
    print(npy_label)
    
    raw_label = np.zeros(np.shape(mel)[1])
    label = np.zeros(np.shape(raw_label))
    print(np.shape(raw_label))
    
    for t_label in npy_label :
        idx_start = int(t_label[0]/time_shift)
        idx_end = int(t_label[1]/time_shift)
        #print(str(idx_start) + ' | ' + str(idx_end))
        raw_label[idx_start:idx_end]=1
    
    # if single shift in frame is activate that block is active
    for i in range(len(raw_label)-3):
        if raw_label[i] or raw_label[i+1] or raw_label[i+2] or raw_label[i+3] :
            label[i]=1
    data = {"mel":torch.from_numpy(mel), "label":torch.from_numpy(label)
        
    ## save
    torch.save(data,'temp.pt')
    

In [25]:
process(0)

(5, 2)
(40, 12737)
[[  4.653   9.008]
 [ 18.976  25.509]
 [ 25.545  30.193]
 [ 35.165  44.951]
 [ 95.33  101.528]]
(12737,)
581 | 1125
2372 | 3188
3193 | 3774
4395 | 5618
11916 | 12691


In [19]:
time_shift

0.008

In [39]:
np.random.seed()
np.random.randint(100)


9

In [69]:
np.random.choice(3,2,replace=False)

array([2, 0])